In [1]:
import pandas as pd
from yahoofinancials import YahooFinancials

selected_date = "2024-04-20"
# selected_date = "2020-09-01"
date = pd.to_datetime(selected_date)
start_date = str((date - pd.Timedelta(days=13)).date())
end_date = str(date.date())
yahoo_financials = YahooFinancials("BTC-USD")
df = yahoo_financials.get_historical_price_data(start_date, end_date, "daily")
prices = df["BTC-USD"]["prices"]
prices

[{'date': 1712448000,
  'high': 70284.4296875,
  'low': 68851.6328125,
  'open': 68897.109375,
  'close': 69362.5546875,
  'volume': 21204930369,
  'adjclose': 69362.5546875,
  'formatted_date': '2024-04-07'},
 {'date': 1712534400,
  'high': 72715.359375,
  'low': 69064.2421875,
  'open': 69362.5546875,
  'close': 71631.359375,
  'volume': 37261432669,
  'adjclose': 71631.359375,
  'formatted_date': '2024-04-08'},
 {'date': 1712620800,
  'high': 71742.5078125,
  'low': 68212.921875,
  'open': 71632.5,
  'close': 69139.015625,
  'volume': 36426900409,
  'adjclose': 69139.015625,
  'formatted_date': '2024-04-09'},
 {'date': 1712707200,
  'high': 71093.4296875,
  'low': 67503.5625,
  'open': 69140.2421875,
  'close': 70587.8828125,
  'volume': 38318601774,
  'adjclose': 70587.8828125,
  'formatted_date': '2024-04-10'},
 {'date': 1712793600,
  'high': 71256.234375,
  'low': 69571.8125,
  'open': 70575.734375,
  'close': 70060.609375,
  'volume': 30153382941,
  'adjclose': 70060.609375,
  '

In [2]:
# Preprocess for prediction
df = pd.DataFrame(prices)
df["date"] = pd.to_datetime(df["formatted_date"])
df.dropna(inplace=True)
df = df.sort_values(by="date")
df.set_index("date", inplace=True)
dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
df = df.reindex(dates, method="ffill")
df.reset_index(inplace=True)
df.rename(columns={"index": "date"}, inplace=True)
df.rename(columns={"volume": "Volume"}, inplace=True)
for i in range(1, 8):
    df[f"lag_close_{i}"] = df["close"].shift(i)
df.dropna(inplace=True)
df

,date,high,low,open,close,Volume,adjclose,formatted_date,lag_close_1,lag_close_2,lag_close_3,lag_close_4,lag_close_5,lag_close_6,lag_close_7
7,2024-04-14,65824.429688,62205.851562,63836.230469,65738.726562,49084320047,65738.726562,2024-04-14,63821.472656,67195.867188,70060.609375,70587.882812,69139.015625,71631.359375,69362.554688
8,2024-04-15,66878.648438,62332.070312,65739.648438,63426.210938,43595917654,63426.210938,2024-04-15,65738.726562,63821.472656,67195.867188,70060.609375,70587.882812,69139.015625,71631.359375
9,2024-04-16,64355.667969,61716.402344,63419.296875,63811.863281,42847528078,63811.863281,2024-04-16,63426.210938,65738.726562,63821.472656,67195.867188,70060.609375,70587.882812,69139.015625
10,2024-04-17,64486.363281,59768.585938,63831.847656,61276.691406,41915247049,61276.691406,2024-04-17,63811.863281,63426.210938,65738.726562,63821.472656,67195.867188,70060.609375,70587.882812
11,2024-04-18,64125.687500,60833.480469,61275.316406,63512.753906,36006307335,63512.753906,2024-04-18,61276.691406,63811.863281,63426.210938,65738.726562,63821.472656,67195.867188,70060.609375
12,2024-04-19,65481.597656,59651.390625,63510.750000,63843.570312,49920425401,63843.570312,2024-04-19,63512.753906,61276.691406,63811.863281,63426.210938,65738.726562,63821.472656,67195.867188
13,2024-04-20,65442.457031,63172.402344,63851.101562,64994.441406,23097485495,64994.441406,2024-04-20,63843.570312,63512.753906,61276.691406,63811.863281,63426.210938,65738.726562,63821.472656


In [3]:
from joblib import load

# Load trained model from a file
try:
    file_name = "trained_models/bitsmart_model_1715639499"
    models = load(file_name)
    model_open = models["open"]
    model_high = models["high"]
    model_low = models["low"]
    model_close = models["close"]
    model_avg = models["average"]
except Exception as e:
    print(e)

c:\Python310\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
def predict_next_7_days(selected_date, data):
    selected_date = pd.to_datetime(selected_date)
    prediction_results = []
    for i in range(7):
        base_date = selected_date - pd.Timedelta(days=i)
        record = data[data["date"] == base_date]
        feature_names = [
            "lag_close_1",
            "lag_close_2",
            "lag_close_3",
            "lag_close_4",
            "lag_close_5",
            "lag_close_6",
            "lag_close_7",
            "Volume",
        ]
        feature_vector = record[feature_names].values.reshape(1, -1)
        feature_vector_df = pd.DataFrame(feature_vector, columns=feature_names)
        predicted_open = model_open.predict(feature_vector_df)[0]
        predicted_high = model_high.predict(feature_vector_df)[0]
        predicted_low = model_low.predict(feature_vector_df)[0]
        predicted_close = model_close.predict(feature_vector_df)[0]
        predicted_average = model_avg.predict(feature_vector_df)[0]
        prediction_results.insert(0,
            {
                "date": (base_date + pd.Timedelta(days=7)).strftime("%Y-%m-%d"),
                "open": predicted_open,
                "high": predicted_high,
                "low": predicted_low,
                "close": predicted_close,
                "average": predicted_average,
            }
        )
    return prediction_results

In [5]:
predictions = predict_next_7_days(selected_date, df)
predictions

[{'date': '2024-04-21',
  'open': 63230.38172993697,
  'high': 67573.0517490519,
  'low': 57917.44354417773,
  'close': 63418.92324171982,
  'average': 63147.41882722234},
 {'date': '2024-04-22',
  'open': 65057.98166454036,
  'high': 68753.02373374887,
  'low': 59750.77469276992,
  'close': 64510.16985157064,
  'average': 64627.142689713466},
 {'date': '2024-04-23',
  'open': 62458.742048285174,
  'high': 66665.35873349143,
  'low': 57867.594550337395,
  'close': 62430.47894497958,
  'average': 62662.316935469236},
 {'date': '2024-04-24',
  'open': 62288.67279519979,
  'high': 66251.20596868868,
  'low': 57417.26934289977,
  'close': 62150.93158072105,
  'average': 62166.285369685545},
 {'date': '2024-04-25',
  'open': 59978.483222831506,
  'high': 63998.75400544481,
  'low': 55181.10193399804,
  'close': 60224.44235049396,
  'average': 60068.42726218178},
 {'date': '2024-04-26',
  'open': 62863.401896090305,
  'high': 66389.02950769961,
  'low': 57727.62849027554,
  'close': 62250.75

In [6]:
def swing_trading_strategy(data, investment_amount):
    if data.empty:
        raise ValueError("No data available")

    # Initialize trading state
    start_date = data["date"][0]
    initial_open_price = data["open"][0]
    bitcoins = investment_amount / initial_open_price
    cash = 0
    sell_executed = False
    load_executed = False
    sell_day = None
    load_day = None
    total_avg_closing_price = 0
    number_of_days = len(data.index)

    for i in range(number_of_days):
        base_date = start_date + pd.Timedelta(days=i)
        price = data[data["date"] == base_date]
        price_high = price["high"].values[0]
        price_low = price["low"].values[0]
        price_avg = price["average"].values[0]

        # Accumulate average closing prices
        total_avg_closing_price += price_avg

        # Decide when to sell: if predicted high price is 5% higher than the open price
        if not sell_executed and price_high > initial_open_price * 1.05:
            sell_executed = True
            sell_day = base_date
            cash = bitcoins * price_high
            bitcoins = 0

        # Decide when to load: after selling, if the predicted low price is 5% lower
        if sell_executed and not load_executed and price_low < price_high * 0.95:
            # Ensure not loading on the sell day
            if base_date != sell_day:
                load_executed = True
                load_day = base_date
                bitcoins = cash / price_low
                cash = 0

    # Calculate average closing price over the prediction period
    average_closing_price = total_avg_closing_price / number_of_days

    # Final valuation at the end of the prediction period or the last available day
    final_day = start_date + pd.Timedelta(days=number_of_days - 1)
    final_close_price = data.loc[data["date"] == final_day, "close"].values[0]
    final_cash = cash if cash > 0 else bitcoins * final_close_price

    # Calculate profit/loss
    profit_loss = final_cash - investment_amount

    return {
        "profit_loss": profit_loss,
        "total_investment_amount": investment_amount,
        "total_exit_amount": final_cash,
        "average_closing_price": average_closing_price,
        "sell_day": str(sell_day.date()) if sell_day else "NA",
        "load_day": str(load_day.date()) if load_day else "NA",
        "final_bitcoins": bitcoins if cash == 0 else 0,
    }

In [7]:
# Preprocess for swing strategy
df = pd.DataFrame(predictions)
df["date"] = pd.to_datetime(df["date"])
df.dropna(inplace=True)
df = df.sort_values(by="date")
df.set_index("date", inplace=True)
dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
df = df.reindex(dates, method="ffill")
df.reset_index(inplace=True)
df.rename(columns={"index": "date"}, inplace=True)
df

,date,open,high,low,close,average
0,2024-04-21,63230.381730,67573.051749,57917.443544,63418.923242,63147.418827
1,2024-04-22,65057.981665,68753.023734,59750.774693,64510.169852,64627.142690
2,2024-04-23,62458.742048,66665.358733,57867.594550,62430.478945,62662.316935
3,2024-04-24,62288.672795,66251.205969,57417.269343,62150.931581,62166.285370
4,2024-04-25,59978.483223,63998.754005,55181.101934,60224.442350,60068.427262
5,2024-04-26,62863.401896,66389.029508,57727.628490,62250.759469,62386.760604
6,2024-04-27,62520.391377,66723.243361,58329.431132,62676.631883,62852.342161


In [8]:
initial_investment = 100000
result = swing_trading_strategy(df, initial_investment)
result

{'profit_loss': 12101.090977174346,
 'total_investment_amount': 100000,
 'total_exit_amount': 112101.09097717435,
 'average_closing_price': 62558.670549973926,
 'sell_day': '2024-04-21',
 'load_day': '2024-04-22',
 'final_bitcoins': 1.7885627802441797}